# Introduction

Takes exploratory data and fits variations of the DDM.

DDM: average-signal

Details: Take every signal that the subject saw in a trial, and average their values. Feed that into a DDM as a time-invariant signal.

## Preamble

In [1]:
# Install (package verification, PyDDM, timer, parallelization)
#!pip3 install paranoid-scientist
#!pip3 install pyddm
#!pip3 install pytictoc  
#!pip3 install pathos  
#!pip3 install pandas
#!pip3 install numpy
#!pip3 install matplotlib

# Python 3.9.13

In [2]:
# Libraries
import os
from pytictoc import TicToc
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import pyddm as ddm
from pyddm import Model, Sample, FitResult, Fittable, Fitted, ICPoint, set_N_cpus
from pyddm.models import NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayUniformMixture, LossRobustBIC
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

In [3]:
# Directories
#datadir = "/Users/ella/Desktop/2023-ddm-sampling-weights/data/processed_data/exploratory.csv"
datadir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-ddm-sampling-weights\\data\\processed_data\\exploratory.csv"
#tempdir = "/Users/ella/Desktop/2023-ddm-sampling-weights/analysis/outputs/temp/"
tempdir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-ddm-sampling-weights\\analysis\\outputs\\temp"

In [4]:
# Parallel settings
ncpu = multiprocessing.cpu_count()-1 # always save one core
ncpu

11

In [5]:
# Do you want a disgusting amount of feedback from model fitting or nah?
verbose_fitting = False

## Import and Clean Raw Data

rawdata_in: odd trials used for in-sample data for model fitting.

rawdata_out: even trials used for out-sample data for model predictions.

In [6]:
# Import data
rawdata = pd.read_csv(datadir)
rawdata = rawdata.rename(columns={"subj":"subject", "rxn_time":"rt"})

# Turn choice into numeric (1=Y, 0=N)
mapping = {"YES":1, "NO":0}
rawdata = rawdata.replace({"choice":mapping})

# Drop no-response trials.
rawdata = rawdata[~np.isnan(rawdata['choice'])]

# Convert rt from ms to s
rawdata.rt = rawdata.rt/1000

# Odd trials -> in-sample. Even trials -> out-sample.
ind = rawdata.trial % 2 != 0
rawdata_in = pd.DataFrame(data=rawdata[ind])
rawdata_out = pd.DataFrame(data=rawdata[~ind])
print(rawdata_in[["subject","trial"]].head(1))
print(rawdata_in[["subject","trial"]].tail(1))
print(rawdata_out[["subject","trial"]].head(1))
print(rawdata_out[["subject","trial"]].tail(1))

   subject  trial
0        1      1
       subject  trial
60659       25    299
   subject  trial
3        1      2
       subject  trial
60669       25    300


---
# Unique Symmetric DDM: unique drift rate for 0, L, and H stimulus values

## Transform data

In [7]:
# Don't mess with the rawdata.
data = pd.DataFrame(data=rawdata_in)

# Transform the data from long to wide, then combine the new stim columns into one tuple.
datawide = data
datawide["stimnum"] = datawide.groupby(["subject","trial"]).cumcount()+1
datalong = datawide.pivot(index=['subject','trial'], columns='stimnum', values='stim')
stim_list = datalong.apply(list, axis=1)
ind = 0
for l in stim_list:
    row = np.array(stim_list.iloc[ind], dtype=np.float32)
    stim_list.iloc[ind] = tuple(row[~np.isnan(row)])
    ind += 1
data = data.groupby(["subject","trial"], as_index=False).first()
data["stim_list"] = stim_list.values
data

,subject,trial,choice,rt,machine,StimuliSeen,stim,S0,S1,S2,Selse,stimnum,stim_list
0,1,1,1.0,0.9739,1,3,2.875,1.125,1.875,2.875,NaN,1,"(2.875, 1.875, 1.125)"
1,1,3,0.0,1.3940,-2,4,0.875,-2.375,-2.875,-2.875,0.875,1,"(0.875, -2.875, -2.875, -2.375)"
2,1,5,0.0,4.0545,-1,13,0.375,-2.625,-2.875,-0.875,0.375,1,"(0.375, 1.375, -2.875, -2.875, 0.125, -0.875, ..."
3,1,7,0.0,1.2802,0,4,0.875,2.875,-0.625,-2.625,0.875,1,"(0.875, -2.625, -0.625, 2.875)"
4,1,9,1.0,5.8468,2,19,0.625,2.875,0.375,1.375,2.125,1,"(0.625, 2.875, 1.625, 0.375, 2.875, 1.375, 2.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3740,25,291,0.0,2.2235,-1,7,2.875,-2.875,-1.625,-2.125,-2.875,1,"(2.875, -1.375, -0.125, -2.875, -2.125, -1.625..."
3741,25,293,0.0,1.5562,-2,5,0.625,-1.625,-2.875,-1.875,-0.125,1,"(0.625, -0.125, -1.875, -2.875, -1.625)"
3742,25,295,0.0,1.2840,0,4,0.375,0.875,-1.375,0.375,0.375,1,"(0.375, 0.375, -1.375, 0.875)"
3743,25,297,0.0,1.6909,2,5,0.125,2.625,0.625,2.875,-0.375,1,"(0.125, -0.375, 2.875, 0.625, 2.625)"


## Fit the Moment-to-Moment DDM

In [8]:
# Create a drift subclass so drift can vary with stimulus.
class Drift(ddm.Drift):
    name = "unique drift rate for zero, low, and high signals"
    BINSIZE = .3 # 300 ms per bin
    required_parameters = ['driftrateZ', 'driftrateL', 'driftrateH'] # How much to scale moment-to-moment drift.
    required_conditions = ['stim_list', 'machine'] # Should be a tuple of values which modulate the moment-to-moment drift.
    def get_drift(self, t, conditions, **kwargs):
        bin_number = int(t//self.BINSIZE) # Which bin are we currently in?
        n_bins = len(conditions['stim_list']) # Total number of bins for this condition.
        # If we are currently in a bin which exceeds the total bins, fix to the slot machine average.
        if bin_number < n_bins:
            signal = conditions['stim_list'][bin_number]
        else:
            signal = conditions['machine']
        # Compute the drift
        if abs(signal) < 0.5:
            return self.driftrateZ * signal
        if abs(signal) >= 0.5 and abs(signal) < 1.5:
            return self.driftrateL * signal
        if abs(signal) >= 1.5:
            return self.driftrateH * signal

# Define the model.
model = Model(name="unique drift rate for zero, low, and high signals",
                 drift=Drift(driftrateZ=Fittable(minval=-1, maxval=4), driftrateL=Fittable(minval=-1, maxval=4), driftrateH=Fittable(minval=-1, maxval=4)),
                 noise=NoiseConstant(noise=Fittable(minval=.001, maxval=2)),
                 bound=BoundConstant(B=1),
                 IC=ICPoint(x0=Fittable(minval=-.99, maxval=.99)),
                 overlay=OverlayNonDecision(nondectime=.1),
                 dx=.01, dt=.01, T_dur=12,  # dx: spatial grid for evidence space (-B to B, in dx bins), dt: time step in s. See Shin et al 2022 Fig 4 for why I set dx=dt.
                 choice_names=("Yes","No"))

In [9]:
# Only run this if specified at the start. Otherwise, just load pre-saved weights.
print("DDM: unique drift rate for stimuli, symmetric about 0")

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:

    # Progress tracker.
    print("============================================================================")
    print("Subject " + str(subnum))

    # Subset the data.
    subdata = data[data["subject"]==subnum]

    # Create a sample object from our data. Sample objects are the standard input for pyDDM fitting functions.
    ddm_data = Sample.from_pandas_dataframe(subdata, rt_column_name="rt", choice_column_name="choice", choice_names=("Yes","No"))

    # Fit the model and show it off. Keep track of how long it took to estimate the parameters.
    clock = TicToc() # Timer
    clock.tic()
    set_N_cpus(ncpu) # Parallelize
    fit_model = fit_adjust_model(sample=ddm_data, model=model,
                                    fitting_method="differential_evolution",
                                    lossfunction=LossRobustBIC,
                                    verbose=verbose_fitting)
    clock.toc("Fitting subject " + str(subnum) + " took")
    display_model(fit_model)

    # Save
    filename = tempdir + "fit_model_us_" + str(subnum) + ".txt"
    with open(filename, "w") as f:
      f.write(repr(fit_model))

DDM: unique drift rate for stimuli, symmetric about 0
Subject 1


KeyboardInterrupt: 

## Extract parameters and BIC for the model objects

In [ ]:
# Placeholders
model_bic = []
model_driftZ = []
model_driftL = []
model_driftH = []
model_noise = []
model_bias = []
model_ndt = []

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:
    
    # Load
    filename = tempdir + "fit_model_us_" + str(subnum) + ".txt"
    with open(filename, "r") as f:
        model_loaded = eval(f.read())

    # Negative Log Likelihood.
    model_bic.append(model_loaded.get_fit_result().value())
    
    # Fitted parameters.
    model_driftZ.append(model_loaded.parameters()['drift']['driftrateZ'])
    model_driftL.append(model_loaded.parameters()['drift']['driftrateL'])
    model_driftH.append(model_loaded.parameters()['drift']['driftrateH'])
    model_noise.append(model_loaded.parameters()['noise']['noise'])
    model_bias.append(model_loaded.parameters()['IC']['x0'])
    model_ndt.append(model_loaded.parameters()['overlay']['nondectime'])
    
d = {'bic':model_bic, "driftZ":model_driftZ, "driftL":model_driftL, "driftH":model_driftH, "noise":model_noise, "bias":model_bias, "ndt":model_ndt}
indiv_model = pd.DataFrame(data=d)
indiv_model

## Means of BIC and Estimates

In [ ]:
summstats_model = pd.DataFrame(data={"mean":indiv_model.mean(), 
                                        "se":indiv_model.sem(),
                                        "ci_lower":indiv_model.mean()-1.96*indiv_model.sem(),
                                        "ci_upper":indiv_model.mean()+1.96*indiv_model.sem()}).T
summstats_model

## Save

In [ ]:
table_model = pd.concat([indiv_model, summstats_model])
filename = tempdir + "table_model_us.txt"
with open(filename, "w") as f:
  f.write(repr(table_model))